# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846838570861                   -0.70    4.8         
  2   -7.852315430888       -2.26       -1.53    1.0   25.6ms
  3   -7.852616688540       -3.52       -2.56    1.5   28.4ms
  4   -7.852645981065       -4.53       -2.89    2.2   33.3ms
  5   -7.852646512537       -6.27       -3.19    1.2   27.5ms
  6   -7.852646679945       -6.78       -4.17    1.0   25.6ms
  7   -7.852646686693       -8.17       -5.28    2.2   34.0ms
  8   -7.852646686727      -10.46       -5.58    1.8   29.9ms
  9   -7.852646686728      -12.32       -5.57    1.0   25.4ms
 10   -7.852646686730      -11.67       -6.76    1.0   25.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846848274848                   -0.70           4.5         
  2   -7.852527894309       -2.25       -1.63   0.80    2.2    295ms
  3   -7.852636773775       -3.96       -2.71   0.80    1.0   24.1ms
  4   -7.852646446548       -5.01       -3.27   0.80    2.0   30.5ms
  5   -7.852646673525       -6.64       -4.10   0.80    1.8   27.6ms
  6   -7.852646686371       -7.89       -4.79   0.80    1.5   27.6ms
  7   -7.852646686724       -9.45       -5.61   0.80    2.0   29.9ms
  8   -7.852646686730      -11.26       -7.01   0.80    1.8   28.7ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.381880e+01     4.022556e+00
 * time: 0.061776161193847656
     1     1.415777e+00     1.937231e+00
 * time: 0.29176902770996094
     2    -1.670444e+00     2.302269e+00
 * time: 0.31676602363586426
     3    -3.794552e+00     1.999406e+00
 * time: 0.35188913345336914
     4    -5.077896e+00     1.753237e+00
 * time: 0.38639116287231445
     5    -6.761117e+00     1.019566e+00
 * time: 0.4212350845336914
     6    -7.401284e+00     5.217440e-01
 * time: 0.45582103729248047
     7    -7.655469e+00     5.235782e-01
 * time: 0.48003315925598145
     8    -7.768748e+00     1.864230e-01
 * time: 0.504249095916748
     9    -7.807246e+00     1.045219e-01
 * time: 0.5285310745239258
    10    -7.833614e+00     7.130251e-02
 * time: 0.5531520843505859
    11    -7.843853e+00     3.853596e-02
 * time: 0.577355146408081
    12    -7.849802e+00     2.316241e-02
 * time: 0.6017360687255859
    13    -7.852006e+00     1.375641e-02
 * time: 0.6259

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846824042206                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645903293                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.08s
  3   -7.852646686730      -13.28       -7.24    178ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.5205839317607363e-7
|ρ_newton - ρ_scfv| = 1.4070457644308792e-7
|ρ_newton - ρ_dm|   = 6.001728832152742e-10
